In [2]:
from tqdm import tqdm
from typing import List, Dict
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from flowprintOptimal.sekigo.core.flowRepresentation import FlowRepresentation,PacketFlowRepressentation,TimeslotRepresentation
from flowprintOptimal.sekigo.dataAnalysis.vNATDataFrameProcessor import VNATDataFrameProcessor
from flowprintOptimal.sekigo.core.flowConfig import FlowConfig
import random
from flowprintOptimal.sekigo.flowUtils.flowDatasets import PacketFlowDataset, BaseFlowDataset, DDQNActivityDataset
from torch.utils.data import Dataset,DataLoader
from torchsampler import ImbalancedDatasetSampler
from sklearn.model_selection import train_test_split
from flowprintOptimal.sekigo.flowUtils.commons import normalizePacketRep
import os
from joblib import Parallel, delayed
from flowprintOptimal.sekigo.flowUtils.commons import saveFlows,loadFlows
from flowprintOptimal.sekigo.dataAnalysis.dataFrameProcessor import UTMobileNetProcessor
from flowprintOptimal.sekigo.flowUtils.dataGetterV2 import getTrainTestOODBalanced, saveTrainTestOODFlows, getFlowReps
from flowprintOptimal.sekigo.flowUtils.conversions import convertPacketRepToTimeslotRepEffecient
from sklearn.metrics import confusion_matrix
import json
from flowprintOptimal.sekigo.modeling.trainers import NNClassificationTrainer
from flowprintOptimal.sekigo.modeling.neuralNetworks import LSTMNetwork,TransformerGenerator,CNNNetwork1D, LSTMDuelingNetwork
from flowprintOptimal.sekigo.modeling.loggers import Logger
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
from flowprintOptimal.sekigo.earlyClassification.DQL.core import MemoryElement,Rewarder,State
from flowprintOptimal.sekigo.earlyClassification.DQL.memoryFiller import MemoryFiller
from flowprintOptimal.sekigo.earlyClassification.DQL.datasets import MemoryDataset
from flowprintOptimal.sekigo.earlyClassification.DQL.trainers import EarlyClassificationtrainer
from flowprintOptimal.sekigo.utils.documentor import Documenter
from flowprintOptimal.sekigo.utils.evaluations import Evaluator,EarlyEvaluation
import warnings
from copy import deepcopy
import torch.nn as nn
warnings.filterwarnings('ignore')

In [3]:
train_packet_flows,test_packet_flows,ood_packet_flows, train_timeslot_flows,test_timeslot_flows,ood_timeslot_flows = getTrainTestOODBalanced(dataset_name= "vnat", test_size= .2, ood_classes= ["FT"],max_flow_length_in_seconds= 240, packet_max_length= 500)

33014
33014
{'streaming': 1582, 'control': 610, 'FT': 459, 'chat': 400}
keep_number = 1582
------------------------------
train class distrubation
streaming    1251
control       490
chat          332
Name: count, dtype: int64
test class distrubation
streaming    331
control      120
chat          68
Name: count, dtype: int64


In [4]:
saveTrainTestOODFlows(base_path= "data/ClassificationOODFlows", dataset_name= "vnat", 
                      train_packet_flows= train_packet_flows, test_packet_flows= test_packet_flows,
                      train_timeslot_flows= train_timeslot_flows, test_timeslot_flows = test_timeslot_flows,
                      ood_packet_flows= ood_packet_flows, ood_timeslot_flows = ood_timeslot_flows
                      )

39433